In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
from nltk.corpus import stopwords, cmudict
from nltk.tokenize import word_tokenize, sent_tokenize
from textblob import TextBlob
import nltk



In [7]:
# Ensure you have the necessary NLTK data files
nltk.download('punkt')
nltk.download('cmudict')
nltk.download('stopwords')

# Initialize cmudict for syllable counting
d = cmudict.dict()

# Load positive and negative words from CSV files
positive_words = set(pd.read_csv('C:/Users/ADMIN/Desktop/Blackcoffer/Loughran-McDonald_MasterDictionary_1993-2023.csv', header=None)[0].tolist())
negative_words = set(pd.read_csv('C:/Users/ADMIN/Desktop/Blackcoffer/Loughran-McDonald_MasterDictionary_1993-2023.csv', header=None)[0].tolist())
personal_pronouns_list = ["i", "we", "my", "ours", "us"]



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_34772\1034588871.py:10: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  positive_words = set(pd.read_csv('C:/Users/ADMIN/Desktop/Blackcoffer/Loughran-McDonald_MasterDictionary_1993-2023.csv', header=None)[0].tolist())
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_34772\1034588871.py:11: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15) have mixed types. Specify dtype option on import or se

In [10]:
# Function to count syllables in a word
def syllable_count(word):
    return len([phoneme for phoneme in d.get(word.lower(), [[word]])[0] if phoneme[-1].isdigit()])

# Functions to compute the required variables
def positive_score(text):
    words = word_tokenize(text)
    return sum(1 for word in words if word.lower() in positive_words)

def negative_score(text):
    words = word_tokenize(text)
    return sum(1 for word in words if word.lower() in negative_words)

def polarity_score(positive, negative):
    return (positive - negative) / ((positive + negative) + 0.000001)

def subjectivity_score(positive, negative, total_words):
    return (positive + negative) / (total_words + 0.000001)

def avg_sentence_length(words, sentences):
    return len(words) / len(sentences)

def percentage_complex_words(words):
    complex_words = [word for word in words if syllable_count(word) >= 3]
    return len(complex_words) / len(words)

def fog_index(avg_sentence_length, percentage_complex_words):
    return 0.4 * (avg_sentence_length + percentage_complex_words * 100)

def avg_words_per_sentence(words, sentences):
    return len(words) / len(sentences)

def complex_word_count(words):
    return sum(1 for word in words if syllable_count(word) >= 3)

def word_count(words):
    stop_words = set(stopwords.words('english'))
    return len([word for word in words if word.lower() not in stop_words])

def syllables_per_word(words):
    return sum(syllable_count(word) for word in words) / len(words)

def personal_pronouns(text):
    pronouns_regex = re.compile(r'\b(?:I|we|my|ours|us)\b', re.IGNORECASE)
    return len(pronouns_regex.findall(text))

def avg_word_length(words):
    return sum(len(word) for word in words) / len(words)

# Read input Excel file
input_df = pd.read_excel('C:/Users/ADMIN/Desktop/Blackcoffer/Input.xlsx')

# Directory to save the articles
os.makedirs('articles', exist_ok=True)

# Function to fetch and save article content
def fetch_and_save_article(url_id, url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Adjust these selectors based on the website structure
        title_element = soup.find('h1')
        article_body_element = soup.find('div', {'class': 'td-post-content'})
        
        if title_element and article_body_element:
            title = title_element.get_text()
            article_body = article_body_element.get_text()
            
            # Save the article text
            with open(f'articles/{url_id}.txt', 'w', encoding='utf-8') as file:
                file.write(title + '\n' + article_body)
        else:
            print(f"Failed to find necessary elements for URL ID {url_id}")
            
    except Exception as e:
        print(f"Failed to process URL ID {url_id}: {e}")

# Iterate over each URL and fetch the content
for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    fetch_and_save_article(url_id, url)

print("Article extraction complete.")

Failed to process URL ID blackassign0036: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Failed to process URL ID blackassign0049: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Article extraction complete.


In [14]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Read input Excel file
input_df = pd.read_excel('C:/Users/ADMIN/Desktop/Blackcoffer/Input.xlsx')

# Directory to save the articles
os.makedirs('articles', exist_ok=True)

# Function to fetch and save article content
def fetch_and_save_article(url_id, url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Adjust these selectors based on the website structure
        title_element = soup.find('h1')
        article_body_element = soup.find('div', {'class': 'td-post-content'})
        
        if title_element and article_body_element:
            title = title_element.get_text()
            article_body = article_body_element.get_text()
            
            # Save the article text
            with open(f'articles/{url_id}.txt', 'w', encoding='utf-8') as file:
                file.write(title + '\n' + article_body)
        else:
            print(f"Failed to find necessary elements for URL ID {url_id}")
            
    except Exception as e:
        print(f"Failed to process URL ID {url_id}: {e}")

# Iterate over each URL and fetch the content
for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    fetch_and_save_article(url_id, url)

print("Article extraction complete.")

# Function to compute sentiment scores using VADER
def compute_vader_scores(text):
    sentiment_dict = analyzer.polarity_scores(text)
    return sentiment_dict['pos'], sentiment_dict['neg'], sentiment_dict['neu'], sentiment_dict['compound']

# Example usage:
text = "VADER is smart, handsome, and funny."
pos, neg, neu, comp = compute_vader_scores(text)
print(f"Positive: {pos}, Negative: {neg}, Neutral: {neu}, Compound: {comp}")


Failed to process URL ID blackassign0036: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Failed to process URL ID blackassign0049: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Article extraction complete.
Positive: 0.746, Negative: 0.0, Neutral: 0.254, Compound: 0.8316


In [16]:
# Read extracted articles and perform analysis
output_data = []

for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    
    try:
        with open(f'articles/{url_id}.txt', 'r', encoding='utf-8') as file:
            text = file.read()
            
        # Clean the text
        cleaned_text = re.sub(r'\s+', ' ', re.sub(r'[^a-zA-Z\s]', '', text)).lower()
        words = word_tokenize(cleaned_text)
        sentences = sent_tokenize(cleaned_text)
        
        pos_score = positive_score(cleaned_text)
        neg_score = negative_score(cleaned_text)
        pol_score = polarity_score(pos_score, neg_score)
        subj_score = subjectivity_score(pos_score, neg_score, len(words))
        avg_sent_len = avg_sentence_length(words, sentences)
        perc_complex_words = percentage_complex_words(words)
        fog_idx = fog_index(avg_sent_len, perc_complex_words)
        avg_words_sent = avg_words_per_sentence(words, sentences)
        complex_word_cnt = complex_word_count(words)
        word_cnt = word_count(words)
        syll_per_word = syllables_per_word(words)
        personal_pronoun_cnt = personal_pronouns(cleaned_text)
        avg_word_len = avg_word_length(words)
        
        output_data.append([
            row['URL_ID'], row['URL'], pos_score, neg_score, pol_score, subj_score, avg_sent_len,
            perc_complex_words, fog_idx, avg_words_sent, complex_word_cnt, word_cnt, syll_per_word,
            personal_pronoun_cnt, avg_word_len
        ])
        
    except Exception as e:
        print(f"Failed to process article for URL ID {url_id}: {e}")

# Define output columns based on the Output Data Structure.xlsx
output_columns = [
    'URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE', 
    'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 
    'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'
]

output_df = pd.DataFrame(output_data, columns=output_columns)
output_df.to_excel('Output nnData Structure.xlsx', index=False)

print("Text analysis complete.")


Failed to process article for URL ID blackassign0036: [Errno 2] No such file or directory: 'articles/blackassign0036.txt'
Failed to process article for URL ID blackassign0049: [Errno 2] No such file or directory: 'articles/blackassign0049.txt'
Text analysis complete.


In [ ]:
import nltk
from nltk.corpus import cmudict, stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from textblob import TextBlob
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import re

# Ensure necessary NLTK data packages are downloaded
nltk.download('punkt')
nltk.download('cmudict')
nltk.download('stopwords')

# Initialize cmudict for syllable counting
d = cmudict.dict()

# Load positive and negative words from text files
with open('C:/Users/ADMIN/Desktop/Blackcoffer/positive-words.txt', 'r') as file:
    positive_words = set(file.read().split())

with open('C:/Users/ADMIN/Desktop/Blackcoffer/negative-words.txt', 'r') as file:
    negative_words = set(file.read().split())

# Function to count syllables in a word
def syllable_count(word):
    return len([phoneme for phoneme in d.get(word.lower(), [[word]])[0] if phoneme[-1].isdigit()])

# Functions to compute the required variables
def positive_score(text):
    return sum(1 for word in word_tokenize(text) if word.lower() in positive_words)

def negative_score(text):
    return sum(1 for word in word_tokenize(text) if word.lower() in negative_words)

def polarity_score(positive, negative):
    return (positive - negative) / ((positive + negative) + 0.000001)

def subjectivity_score(positive, negative, total_words):
    return (positive + negative) / (total_words + 0.000001)

def avg_sentence_length(words, sentences):
    return len(words) / len(sentences)

def percentage_complex_words(words):
    complex_words = [word for word in words if syllable_count(word) >= 3]
    return len(complex_words) / len(words)

def fog_index(avg_sentence_length, percentage_complex_words):
    return 0.4 * (avg_sentence_length + percentage_complex_words * 100)

def avg_words_per_sentence(words, sentences):
    return len(words) / len(sentences)

def complex_word_count(words):
    return sum(1 for word in words if syllable_count(word) >= 3)

def word_count(words):
    stop_words = set(stopwords.words('english'))
    return len([word for word in words if word.lower() not in stop_words])

def syllables_per_word(words):
    return sum(syllable_count(word) for word in words) / len(words)

def personal_pronouns(text):
    pronouns_regex = re.compile(r'\b(?:I|we|my|ours|us)\b', re.IGNORECASE)
    return len(pronouns_regex.findall(text))

def avg_word_length(words):
    return sum(len(word) for word in words) / len(words)

# Read input Excel file
input_df = pd.read_excel('C:/Users/ADMIN/Desktop/Blackcoffer/Input.xlsx')

# Directory to save the articles
os.makedirs('articles', exist_ok=True)

for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Adjust these selectors based on the website structure
        title_element = soup.find('h1')
        article_body_element = soup.find('div', {'class': 'td-post-content'})
        
        if title_element and article_body_element:
            title = title_element.get_text()
            article_body = article_body_element.get_text()
            
            # Save the article text
            with open(f'articles/{url_id}.txt', 'w', encoding='utf-8') as file:
                file.write(title + '\n' + article_body)
        else:
            print(f"Failed to find necessary elements for URL ID {url_id}")
            
    except Exception as e:
        print(f"Failed to process URL ID {url_id}: {e}")

print("Article extraction complete.")

# Processing the articles to calculate the required metrics
results = []

for filename in os.listdir('articles'):
    if filename.endswith('.txt'):
        with open(f'articles/{filename}', 'r', encoding='utf-8') as file:
            text = file.read()
            
        words = word_tokenize(text)
        sentences = sent_tokenize(text)
        
        positive = positive_score(text)
        negative = negative_score(text)
        polarity = polarity_score(positive, negative)
        subjectivity = subjectivity_score(positive, negative, len(words))
        avg_sentence_len = avg_sentence_length(words, sentences)
        pct_complex_words = percentage_complex_words(words)
        fog_idx = fog_index(avg_sentence_len, pct_complex_words)
        avg_words_sentence = avg_words_per_sentence(words, sentences)
        complex_words_cnt = complex_word_count(words)
        word_cnt = word_count(words)
        syllables_word = syllables_per_word(words)
        personal_pronouns_cnt = personal_pronouns(text)
        avg_word_len = avg_word_length(words)
        
        results.append({
            'URL_ID': filename.split('.')[0],
            'POSITIVE SCORE': positive,
            'NEGATIVE SCORE': negative,
            'POLARITY SCORE': polarity,
            'SUBJECTIVITY SCORE': subjectivity,
            'AVG SENTENCE LENGTH': avg_sentence_len,
            'PERCENTAGE OF COMPLEX WORDS': pct_complex_words,
            'FOG INDEX': fog_idx,
            'AVG NUMBER OF WORDS PER SENTENCE': avg_words_sentence,
            'COMPLEX WORD COUNT': complex_words_cnt,
            'WORD COUNT': word_cnt,
            'SYLLABLES PER WORD': syllables_word,
            'PERSONAL PRONOUNS': personal_pronouns_cnt,
            'AVG WORD LENGTH': avg_word_len
        })

# Convert results to DataFrame and save to Excel
output_df = pd.DataFrame(results)
output_df.to_excel('/mnt/data/Output_Data_Structure.xlsx', index=False)

print("Analysis complete and results saved to 'Output_Data_Structure.xlsx'.")
